In [ ]:
# Import dependencies
import pandas as pd
import spacy # nlp library to filer out names that are not people

# custom functions
from src.clean_names_helpers import remove_non_person_contributors_and_export
from src.clean_names_helpers import format_name_to_lastname_initials

In [ ]:
# Load the multilingual NER model
nlp = spacy.load("xx_ent_wiki_sm")

In [ ]:
# Initialize values
NROWS = None # None for all

# names that spacy does not recognize as such
# NOTE: Add the verbatim names here, not the standardized target notation 
# This list can be fed from removed_contributors.csv that is created when running the script
WHITELIST = [ 
    "Oosterlaan, J.",
    "Nollet, F."
    ] 

# non-people's names that don't get filtered out by spacy 
BLACKLIST = [
    "Cardiology"
]

removed_contributors = []

In [ ]:
# Read data
pairs_raw = pd.read_csv("data/raw/pairs_sups_phds.csv", nrows=NROWS)
pairs_raw = pairs_raw.convert_dtypes() # make sure all integer columns are integer dtype 
pairs_raw.head()

In [ ]:
# remove duplicates
pairs_filtered = pairs_raw.drop_duplicates() 

# Remove rows where 'contributor' is NA
pairs_filtered = pairs_filtered.dropna(subset=['contributor'])

# remove contributors that aren't people
csv_path = "data/removed_contributors.csv"
pairs_filtered = remove_non_person_contributors_and_export(pairs_filtered, csv_path, nlp, WHITELIST, BLACKLIST)
len(pairs_filtered)

In [ ]:
# Standardize names
pairs_std = pairs_filtered
# Apply name standardization to the contributor column
pairs_std['contributor'] = pairs_filtered['contributor'].apply(format_name_to_lastname_initials)

len(pairs_std)

In [ ]:
# Group by publication
aggregated = pairs_std.groupby([
        'integer_id', 
        'thesis_identifier', 
        'institution', 
        'author_name', 
        'title', 
        'year', 
        'language'
    ])
        
# Aggregate contributors into a list
aggregated = aggregated.agg(list)

aggregated = aggregated.reset_index()
    
# make sure the contributor is a sequence from 1 to n_contributors
aggregated['contributor_order'] = aggregated['contributor_order'].apply(lambda lst: list(range(1, len(lst) + 1)))


In [ ]:
# Pivor the dataset, to get to one row per dissertation, with the contributors in columns

# Initialize a list to hold publication data dictionaries
pubs_list = []

# Iterate over each aggregated group
for _, row in aggregated.iterrows():
    # Initialize a dictionary with publication information
    pub_dict = {col: row[col] for col in ['integer_id', 'thesis_identifier', 'institution', 'author_name', 'title', 'year', 'language']}
    
    # Get the list of contributors and their orders for this publication
    contributors = row['contributor']
    contributor_orders = row['contributor_order']
    
    # Add contributors to the dictionary using dynamic keys
    for order in sorted(set(contributor_orders)):  # Ensure unique and sorted order numbers
        if order - 1 < len(contributors):  # Check to prevent index error
            pub_dict[f'contributor_{order}'] = contributors[order - 1]
    
    # Append the publication dictionary to the list
    pubs_list.append(pub_dict)

# Convert the list of dictionaries to a DataFrame
pubs = pd.DataFrame(pubs_list)

# Ensure correct data types and fill missing values with a suitable placeholder if necessary
pubs = pubs.convert_dtypes()

pubs.head()

In [ ]:
# Export a sample of the dataset to easily share it
seed = 42 # fixed seed

# Sample 100 random lines from the DataFrame
sampled_pubs = pubs.sample(n=100, random_state=seed)

# Export the sampled DataFrame to a CSV file
sampled_pubs.to_csv('sampled_pubs.csv', index=False)